In [61]:
import pandas as pd
import difflib

In [62]:
preprocessed = pd.read_csv('data/usefulrecipecalories.csv')[['calories', 'parsed_ingredients']]
ing = pd.read_csv('data/ingredientcalories.csv')
check = ing['FoodItem'].str.lower()

In [82]:
def get_cals(ing_list):
  ing_list = ing_list.split(',')
  lower_list = list(map(str.lower, ing_list))
  new_ing_list = []
  for i in range(len(lower_list)):
    if lower_list[i] not in check:
      new_ing = difflib.get_close_matches(lower_list[i], check, n=1)
      if (len(new_ing) > 0):
        new_ing_list.append(new_ing[0])
    else:
      new_ing_list.append(lower_list[i])
  cleaned = ing[check.isin(new_ing_list)].drop_duplicates('FoodItem')
  cleaned['Cals_per100grams'] = cleaned['Cals_per100grams'].map(lambda x : eval(x[:-4]))
  cleaned.sort_values(by=['Cals_per100grams'], inplace=True, ascending=False)
  ret = cleaned['Cals_per100grams'].to_list()
  return ret

In [83]:
preprocessed['parsed_ingredients'] = preprocessed['parsed_ingredients'].map(get_cals)

In [85]:
preprocessed.to_csv('data/trainingdata.csv')